In [1]:
import torch
from torch.utils.data import DataLoader
from dataset import RTB
import torch.nn as nn
import torch.nn.functional as F
from model import ProbabilityANN
import torch.optim as optim
from tqdm import tqdm
import os

imp_file_path = "Adobe_dataset/imp.06.txt"
clk_file_path = "Adobe_dataset/clk.06.txt"
conv_file_path = "Adobe_dataset/conv.06.txt"

train_dataset = RTB(imp_file_path,clk_file_path,conv_file_path)

Key 16751 not in column 
Attribute : Long-term interest/sex

imp_train shape: (90753, 53)
imp_val shape: (1724322, 53)
clk_train shape: (90753,)
clk_val shape: (1724322,)
conv_train shape: (90753,)
conv_val shape: (1724322,)


In [ ]:
from model import ProbabilityANN
from torchvision.ops.focal_loss import sigmoid_focal_loss

device = "cuda" if torch.cuda.is_available() else "cpu"

batch_size = 10000
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

model = ProbabilityANN().to(device)

weights_folder = f"weights"
os.makedirs(weights_folder, exist_ok=True)

optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 50

In [ ]:
print(len(train_loader))

In [ ]:
for epoch in tqdm(range(epochs), desc="Epoch : "):
    
    running_loss = 0
    progress_bar = tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}", position=0, leave=True)
    
    for i, data in enumerate(train_loader, 0):
        
        inputs, clk_label, conv_label = data[0].to(device), data[1].to(device), data[2].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
       
        clk_prob , conv_prob = outputs[:,0] , outputs[:,1]
        
        loss1 = sigmoid_focal_loss(clk_prob, clk_label, alpha=0.9993, gamma=2, reduction="mean")
        loss2 = sigmoid_focal_loss(conv_prob, conv_label, alpha=0.9996, gamma=2, reduction="mean")

        loss = loss1 + loss2
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss
        
        progress_bar.set_postfix(running_loss=running_loss / (i+1))
        progress_bar.update(1)
    
        # if(i % ((len(train_dataset)//batch_size)//10) == 0):
        #     print(f"Batch {i+1} | Epoch Loss {running_loss} | Iteration Loss {loss.item()}")
        
        # if(i % ((len(train_dataset)//batch_size)//5) == 0):
        #     weights_file_path = f"{weights_folder}/Epoch_{epoch+1}_Iteration_{i}_loss_{loss}.pt"
        #     torch.save(model.state_dict(), weights_file_path)
        
        # iteration_tqdm.set_postfix(loss=loss.item())
    
    progress_bar.close()    
    
    print(f"Epoch {epoch+1} | Loss {running_loss} ")
    weights_file_path = f"{weights_folder}/Epoch_{epoch+1}_Loss_{running_loss}.pt"
    # torch.save(model.state_dict(), weights_file_path)
    
    with torch.no_grad():
        